In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Stroke"
cohort = "GSE38571"

# Input paths
in_trait_dir = "../../input/GEO/Stroke"
in_cohort_dir = "../../input/GEO/Stroke/GSE38571"

# Output paths
out_data_file = "../../output/preprocess/Stroke/GSE38571.csv"
out_gene_data_file = "../../output/preprocess/Stroke/gene_data/GSE38571.csv"
out_clinical_data_file = "../../output/preprocess/Stroke/clinical_data/GSE38571.csv"
json_path = "../../output/preprocess/Stroke/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Integrated transcriptomic and epigenomic analysis of primary human lung cell differentiation"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['time: D6', 'time: D4', 'time: D0', 'time: D2', 'time: D8'], 1: ['cell type: AT cell', 'cell type: AT cell (AT2)', 'cell type: AT cell (AT1-like)'], 2: ['Sex: female'], 3: ['age (y): 49', 'age (y): 61', 'age (y): 66'], 4: ['smoker: non-smoker'], 5: ['cod: Anoxia', 'cod: CVA-Stroke', 'cod: ICH-Stroke']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset is about cell differentiation
# and includes transcriptomic analysis, which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary
# No direct mention of stroke in the dictionary, but we may need to infer it
# Time and cell type information is available, but no direct stroke information
trait_row = None  # Stroke data is not available in this dataset

# Age information is not present in the sample characteristics
age_row = None

# Gender is available at key 2, but it shows only "Sex: male" which indicates a constant value
gender_row = None  # Although gender is mentioned, it's constant (only male)

# 2.2 Data Type Conversion Functions
# Since trait data is not available, we still define a conversion function for completeness
def convert_trait(value):
    if value is None:
        return None
    value = value.lower() if isinstance(value, str) else str(value).lower()
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Binary conversion for stroke status
    if 'stroke' in value or 'case' in value:
        return 1
    elif 'control' in value or 'healthy' in value or 'normal' in value:
        return 0
    return None

# Age conversion function - not needed but defined for completeness
def convert_age(value):
    if value is None:
        return None
    value = value.lower() if isinstance(value, str) else str(value).lower()
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        # Try to convert to float for continuous age
        return float(value)
    except:
        return None

# Gender conversion function - not needed but defined for completeness
def convert_gender(value):
    if value is None:
        return None
    value = value.lower() if isinstance(value, str) else str(value).lower()
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Binary conversion for gender
    if 'female' in value or 'f' in value:
        return 0
    elif 'male' in value or 'm' in value:
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability (is_trait_available)
is_trait_available = trait_row is not None

# Validate and save cohort information - initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, skip this substep


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Stroke/GSE38571/GSE38571-GPL10558_series_matrix.txt.gz
Gene data shape: (47231, 17)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers are Illumina probe IDs (starting with ILMN_), not human gene symbols
# They need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'GB_ACC', 'SPOT_ID']
{'ID': ['ILMN_1356720', 'ILMN_1355539', 'ILMN_1365415', 'ILMN_1373448', 'ILMN_1353631'], 'Species': ['Rattus norvegicus', 'Rattus norvegicus', 'Rattus norvegicus', 'Rattus norvegicus', 'Rattus norvegicus'], 'Source': ['RefSeq', 'RefSeq', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['GI_62644958-S', 'GI_62643181-S', 'GI_20301967-S', 'GI_62647669-S', 'GI_62658996-S'], 'Transcript': ['ILMN_57573', 'ILMN_58017', 'ILMN_297955', 'ILMN_54533', 'ILMN_289444'], 'ILMN_Gene': ['LOC499782', 'LOC502515', 'PRSS8', 'WBP1', 'COX6A1'], 'Sou

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in gene_annotation that store probe IDs and gene symbols
# From the previous output, we can see:
# - 'ID' column contains the Illumina probe IDs (e.g., ILMN_1356720)
# - 'Symbol' column contains the gene symbols (e.g., LOC499782, Prss8, Cox6a1)

# However, there's an important observation - the species is "Rattus norvegicus" (rat)
# rather than human, so we need to be careful about interpreting these results

# 2. Get a gene mapping dataframe by extracting probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("Gene mapping preview:")
print(mapping_df.head())

# Check how many probes map to genes (non-empty mapping)
print(f"Number of probes with gene symbol mapping: {len(mapping_df)}")

# 3. Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])

# Save the processed gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# Set final metadata and conclude
print("\nNote: This dataset (GSE38571) contains rat gene expression data, not human data.")
print("This is not suitable for human stroke studies.")


Gene mapping dataframe shape: (23190, 2)
Gene mapping preview:
             ID       Gene
0  ILMN_1356720  LOC499782
1  ILMN_1355539  LOC502515
2  ILMN_1365415      Prss8
3  ILMN_1373448       Wbp1
4  ILMN_1353631     Cox6a1
Number of probes with gene symbol mapping: 23190
Gene expression data shape after mapping: (0, 17)
First few gene symbols:
Index([], dtype='object', name='Gene')
Gene expression data saved to ../../output/preprocess/Stroke/gene_data/GSE38571.csv

Note: This dataset (GSE38571) contains rat gene expression data, not human data.
This is not suitable for human stroke studies.


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
# Use the gene_data variable that was already loaded in Step 4
print(f"Original gene data shape: {gene_data.shape}")

# Normalize gene symbols
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data_normalized.shape}")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. No clinical data available (trait_row is None from previous steps)
# We can't proceed with linking clinical and genetic data
print("No trait data available. Can't proceed with linking clinical and genetic data.")

# Create a minimal dataframe for validation purposes 
# (since we need a valid DataFrame when is_final=True)
empty_df = pd.DataFrame({"dummy": [0]})

# Since trait data is not available, the dataset is not usable for our purposes
# We pass is_biased=True to indicate unusable data
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,
    is_biased=True,  # Setting is_biased to True since missing trait data makes dataset unusable
    df=empty_df,
    note="No stroke-specific trait data available in this dataset. Contains gene expression data from peripheral blood of older adults, but without clear stroke indicators."
)
print("Dataset deemed not usable due to missing trait data.")

Original gene data shape: (0, 17)
Gene data shape after normalization: (0, 17)
Normalized gene expression data saved to ../../output/preprocess/Stroke/gene_data/GSE38571.csv
No trait data available. Can't proceed with linking clinical and genetic data.
Abnormality detected in the cohort: GSE38571. Preprocessing failed.
Dataset deemed not usable due to missing trait data.
